In [2]:
import pandas as pd
import openpyxl
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors
import time
import datetime
from datetime import date

In [4]:
name_file_abiturs = 'data/abitur.xlsx'
name_file_person = 'data/person.xlsx'

In [17]:
df_abitur = pd.read_excel(name_file_abiturs,skiprows=3,usecols=['Абитуриент','Доп. статус','Состояние'])
df_person = pd.read_excel(name_file_person,sheet_name='Абитуриенты',skiprows=8,usecols=['ФИО','Нуждается в общежитии','Формирующее подр.','Направление, специальность, профессия','Сдан оригинал'])

In [18]:
df_person.head()

,ФИО,Нуждается в общежитии,Формирующее подр.,"Направление, специальность, профессия",Сдан оригинал
0,Абидуев Александр Валерьевич,да,Отделение железнодорожного транспорта,23.02.06 Техническая эксплуатация подвижного с...,да
1,Аввакумов Виталий Владимирович,нет,Отделение железнодорожного транспорта,23.02.06 Техническая эксплуатация подвижного с...,да
2,Аввакумов Виталий Владимирович,нет,Отделение железнодорожного транспорта,23.01.10 Слесарь по обслуживанию и ремонту под...,да
3,Аверина Валерия Станиславовна,нет,Отделение железнодорожного транспорта,43.02.06 Сервис на транспорте (по видам трансп...,нет
4,Агафонов Максим Сергеевич,нет,Хоринский филиал,23.01.17 Мастер по ремонту и обслуживанию авто...,да


In [16]:
df_abitur.head()

,Абитуриент,Доп. статус,Состояние
0,Черанева Яна Николаевна,NaN,Зачислен
1,Веприкова Виктория Сергеевна,NaN,Зачислен
2,Коноплева Ольга Викторовна,NaN,Зачислен
3,Чупышев Богдан Станиславович,NaN,Зачислен
4,Бильдаков Аюр Анатольевич,NaN,Зачислен


In [22]:
df_person.shape

(791, 5)

In [23]:
df_abitur.shape

(650, 3)

In [ ]:
# Соединяем оба датафрейма

In [19]:
cross_df = df_person.merge(df_abitur,how='inner',left_on='ФИО',right_on='Абитуриент')

In [20]:
cross_df

,ФИО,Нуждается в общежитии,Формирующее подр.,"Направление, специальность, профессия",Сдан оригинал,Абитуриент,Доп. статус,Состояние
0,Абидуев Александр Валерьевич,да,Отделение железнодорожного транспорта,23.02.06 Техническая эксплуатация подвижного с...,да,Абидуев Александр Валерьевич,NaN,Зачислен
1,Аввакумов Виталий Владимирович,нет,Отделение железнодорожного транспорта,23.02.06 Техническая эксплуатация подвижного с...,да,Аввакумов Виталий Владимирович,NaN,Зачислен
2,Аввакумов Виталий Владимирович,нет,Отделение железнодорожного транспорта,23.01.10 Слесарь по обслуживанию и ремонту под...,да,Аввакумов Виталий Владимирович,NaN,Зачислен
3,Аверина Валерия Станиславовна,нет,Отделение железнодорожного транспорта,43.02.06 Сервис на транспорте (по видам трансп...,нет,Аверина Валерия Станиславовна,NaN,Забрал документы
4,Агафонов Максим Сергеевич,нет,Хоринский филиал,23.01.17 Мастер по ремонту и обслуживанию авто...,да,Агафонов Максим Сергеевич,NaN,Зачислен
...,...,...,...,...,...,...,...,...
790,Юй-Най-Ли Владимир Евгеньевич,нет,Отделение металлообработки,15.01.35 Мастер слесарных работ,да,Юй-Най-Ли Владимир Евгеньевич,NaN,Зачислен
791,Юмдылыков Алдар Сангажапович,нет,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,да,Юмдылыков Алдар Сангажапович,NaN,Зачислен
792,Яковлев Вадим Михайлович,нет,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,нет,Яковлев Вадим Михайлович,NaN,Зачислен
793,Яковлев Константин Иванович,да,NaN,NaN,нет,Яковлев Константин Иванович,NaN,NaN


In [38]:
# Преобразовываем да-нет в 1 или 0 для подсчетов
cross_df['Нуждается в общежитии'] =cross_df['Нуждается в общежитии'].apply(lambda x:0 if x =='нет' else 1)
cross_df['Сдан оригинал'] =cross_df['Сдан оригинал'].apply(lambda x:0 if x =='нет' else 1)
cross_df['Состояние'] =cross_df['Состояние'].apply(lambda x:1 if x =='Забрал документы' else 0)


In [39]:
# заменяем нан на пустые строки чтобы произвести поиск слова сирота;
cross_df['Доп. статус'].fillna('',inplace=True)
cross_df['Доп. статус'] = cross_df['Доп. статус'].apply(lambda x:1 if 'Сирота;' in x else 0)

In [40]:
cross_df['for_counting'] = 1

In [41]:
cross_df

,ФИО,Нуждается в общежитии,Формирующее подр.,"Направление, специальность, профессия",Сдан оригинал,Абитуриент,Доп. статус,Состояние,for_counting
0,Абидуев Александр Валерьевич,1,Отделение железнодорожного транспорта,23.02.06 Техническая эксплуатация подвижного с...,1,Абидуев Александр Валерьевич,0,0,1
1,Аввакумов Виталий Владимирович,1,Отделение железнодорожного транспорта,23.02.06 Техническая эксплуатация подвижного с...,1,Аввакумов Виталий Владимирович,0,0,1
2,Аввакумов Виталий Владимирович,1,Отделение железнодорожного транспорта,23.01.10 Слесарь по обслуживанию и ремонту под...,1,Аввакумов Виталий Владимирович,0,0,1
3,Аверина Валерия Станиславовна,1,Отделение железнодорожного транспорта,43.02.06 Сервис на транспорте (по видам трансп...,1,Аверина Валерия Станиславовна,0,0,1
4,Агафонов Максим Сергеевич,1,Хоринский филиал,23.01.17 Мастер по ремонту и обслуживанию авто...,1,Агафонов Максим Сергеевич,0,0,1
...,...,...,...,...,...,...,...,...,...
790,Юй-Най-Ли Владимир Евгеньевич,1,Отделение металлообработки,15.01.35 Мастер слесарных работ,1,Юй-Най-Ли Владимир Евгеньевич,0,0,1
791,Юмдылыков Алдар Сангажапович,1,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,1,Юмдылыков Алдар Сангажапович,0,0,1
792,Яковлев Вадим Михайлович,1,Отделение железнодорожного транспорта,23.01.09 Машинист локомотива,1,Яковлев Вадим Михайлович,0,0,1
793,Яковлев Константин Иванович,1,NaN,NaN,1,Яковлев Константин Иванович,0,0,1


In [42]:
pd.DataFrame.pivot_table(cross_df,index=['Формирующее подр.','Направление, специальность, профессия'],
                     values=['for_counting','Состояние','Сдан оригинал','Доп. статус','Нуждается в общежитии'],
                     aggfunc='sum')

for_counting  \
Формирующее подр.                     Направление, специальность, профессия                              
Отделение железнодорожного транспорта 23.01.09 Машинист локомотива                                 114   
                                      23.01.10 Слесарь по обслуживанию и ремонту подв...            69   
                                      23.02.06 Техническая эксплуатация подвижного со...            31   
                                      43.01.06 Проводник на железнодорожном транспорте              52   
                                      43.02.06 Сервис на транспорте (по видам транспо...            39   
Отделение металлообработки            15.01.05 Сварщик (ручной и частично механизиров...            75   
                                      15.01.33 Токарь на станках с числовым программн...            45   
                                      15.01.35 Мастер слесарных работ                               45   
                                      23.01.17 Мастер по ремонту и обслуживанию автом...            61   
Отделение энергоснабжения             08.02.08 Монтаж и эксплуатация оборудования и с...            33   
                                      08.02.09 Монтаж, наладка и эксплуатация электро...            46   
                                      13.01.10 Электромонтер по ремонту и обслуживани...            29   
                                      23.02.07 Техническое обслуживание и ремонт двиг...            54   
                                      38.02.02 Страховое дело (по отраслям)                         44   
Хоринский филиал                      23.01.17 Мастер по ремонту и обслуживанию автом...            31   
                                      43.01.09 Повар, кондитер                                      21   

                                                                                          Доп. статус  \
Формирующее подр.                     Направление, специальность, профессия                             
Отделение железнодорожного транспорта 23.01.09 Машинист локомотива                                  4   
                                      23.01.10 Слесарь по обслуживанию и ремонту подв...            0   
                                      23.02.06 Техническая эксплуатация подвижного со...            2   
                                      43.01.06 Проводник на железнодорожном транспорте              2   
                                      43.02.06 Сервис на транспорте (по видам транспо...            1   
Отделение металлообработки            15.01.05 Сварщик (ручной и частично механизиров...            4   
                                      15.01.33 Токарь на станках с числовым программн...            2   
                                      15.01.35 Мастер слесарных работ                               3   
                                      23.01.17 Мастер по ремонту и обслуживанию автом...            6   
Отделение энергоснабжения             08.02.08 Монтаж и эксплуатация оборудования и с...            5   
                                      08.02.09 Монтаж, наладка и эксплуатация электро...            1   
                                      13.01.10 Электромонтер по ремонту и обслуживани...            1   
                                      23.02.07 Техническое обслуживание и ремонт двиг...            0   
                                      38.02.02 Страховое дело (по отраслям)                         1   
Хоринский филиал                      23.01.17 Мастер по ремонту и обслуживанию автом...            1   
                                      43.01.09 Повар, кондитер                                      0   

                                                                                          Нуждается в общежитии  \
Формирующее подр.                     Направление, специальность, профессия                                       
Отделение железнодорожного транспорта 23.01.09 Машинист локо